In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from tqdm import tqdm 
import time
from scipy.optimize import curve_fit
import random
import datetime
import yfinance as yf
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

## Projeto de Webscrapping - análise de ETFs
Vamos construir um código que vá no site etf.com e busque dados de todos os etfs do mercado mundial. Rentabilidaddes, patrimônio, gestora, taxa...

#### Será necessário:
- definir navegador a ser utilizado
- entender requisições na internet
- conhecer e mapear o processo de coleta de dados no site ETF.com
- encontrar os elementos necessarios dentro do HTML
- construir tabela final

In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

- Como funcionam requisições na internet

HTTP é um protocolo (protocol) que permite a obtenção de recursos, como documentos HTML. É a base de qualquer troca de dados na Web e um protocolo cliente-servidor, o que significa que as requisições são iniciadas pelo destinatário, geralmente um navegador da Web. Um documento completo é reconstruído a partir dos diferentes sub-documentos obtidos, como por exemplo texto, descrição do layout, imagens, vídeos, scripts e muito mais.

Clientes e servidores se comunicam trocando mensagens individuais (ao contrário de um fluxo de dados). As mensagens enviadas pelo cliente, geralmente um navegador da Web, são chamadas de solicitações (requests), ou também **requisições**, e as mensagens enviadas pelo servidor como resposta são chamadas de respostas (responses).

O HTTP é um protocolo cliente-servidor: as requisições são enviados por uma entidade, o agente-usuário (ou um proxy em nome dele). A maior parte do tempo, o agente-usuário é um navegador da Web, mas pode ser qualquer coisa, como por exemplo um robô que varre a Web para preencher e manter um índice de mecanismo de pesquisa e coletar informações.

Cada requisição individual é enviada para um servidor, que irá lidar com isso e fornecer um resultado, chamado de resposta. Entre a solicitação e a resposta existem várias entidades, designadas coletivamente como proxies, que executam operações diferentes e atuam como gateways (intermediários) ou caches, por exemplo.

Na realidade, existem muitos outros computadores entre o navegador e o servidor que está tratando a requisição: existem roteadores, modems e muito mais. Graças ao modelo de camadas da Web, essas funcionalidades estão escondidas nas camadas de rede e transporte, respectivamente. O HTTP está no topo da camada de aplicação.

*fonte: https://developer.mozilla.org/pt-BR/docs/Web/HTTP/Overview *

Ao acessar qualquer site, o computador pede um get() e o servidor deve autorizar. Quando estamos fazendo um login em algum site, estamos fazendo um post(), e esperamos uma resposta (no caso, sobre a validade da senha).

In [3]:
driver = webdriver.Firefox( service=FirefoxService(GeckoDriverManager().install())
)#nosso bot que irá abrir o navegador e procurar os sites

[WDM] - Downloading: 16.9kB [00:00, 2.96MB/s]                   


In [4]:
#Ex1
url = "https:www.google.com"
driver.get(url)

In [5]:
url = "https://www.etf.com/etfanalytics/etf-finder"
driver.get(url)

In [6]:
'''
plano: o bot abrir a pagina, mudar a visualizar 100, copiar toda a tabela, 
guardar, e passar a página, 
copiar a tabela...
(OBS: copy full xpath)
'''
time.sleep(7) #Python é mais rapido que o carregamento da tabela
# achar botao viz100
botao_100 = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/div/div/
div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')
#clicar
driver.execute_script("arguments[0].click()", botao_100)

#encontrar o numero total de paginas para fazer loop
numero_paginas = driver.find_element("xpath",'''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]
''')
print(numero_paginas.text)


of 32


In [7]:
#pegando elemento numerico
numero_paginas = int(numero_paginas.text.replace("of",""))
numero_paginas

32

In [8]:
#criando loop para ler a tabela de dados (ler rentabilidade)
html_tabela = driver.find_element("xpath", '''
/html/body/div[5]/section/div/div[3]/section/div/div/div/
div/div[2]/section[2]/div[2]/div/table
''')

html_tabela = html_tabela.get_attribute("outerHTML")
#Agora leremos HTML com pandas :D
tabela = pd.read_html(html_tabela)[0] #recortar primeira p ignorar outros elementos html
tabela

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B
...,...,...,...,...,...,...
95,VXF,Vanguard Extended Market ETF,Equity: U.S. - Extended Market,Vanguard,0.06%,$14.69B
96,XLI,Industrial Select Sector SPDR Fund,Equity: U.S. Industrials,State Street Global Advisors,0.10%,$14.15B
97,PFF,iShares Preferred and Income Securities ETF,"Fixed Income: U.S. - Corporate, Preferred",Blackrock,0.45%,$13.97B
98,VGIT,Vanguard Intermediate-Term Treasury Index ETF,"Fixed Income: U.S. - Government, Treasury Inve...",Vanguard,0.04%,$13.95B


In [9]:
#Guardar as informações da tabela da primeira pagina e avançar de pagina
tabelas = []

for i in range(0,numero_paginas):
    html_tabela = driver.find_element("xpath", '''
/html/body/div[5]/section/div/div[3]/section/div/div/div/
div/div[2]/section[2]/div[2]/div/table
''')

    html_tabela = html_tabela.get_attribute("outerHTML")
#Agora leremos HTML com pandas :D
    tabela = pd.read_html(html_tabela)[0] #recortar primeira p ignorar outros elementos html
    tabelas.append(tabela)
#Encontrar o botao de avancar pagina e clicar
    avancar = driver.find_element("xpath", "/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]")
    driver.execute_script("arguments[0].click();", avancar)

In [10]:
#Tranformar a lista de dataframes num grande dataframe

In [11]:
## Tutorial de concatenação de dataframes:

df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
)


df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
)

pd.concat([df1, df2], ignore_index = True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [12]:
pd.concat([df1, df2], ignore_index = True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [13]:
tabelas = pd.concat(tabelas)

In [14]:
tabelas

,Ticker,Name,Segment,Issuer,Expense Ratio,AUM
0,SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B
1,IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B
2,VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B
3,VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B
4,QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B
...,...,...,...,...,...,...
8,CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--
9,CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--
10,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--
11,CVIE,Calvert International Responsible Index ETF,Equity: Developed Markets Ex-U.S. - Large Cap,Morgan Stanley,--,--


In [15]:
### Lendo a tabela de performance
#botao performance
botao_aba = driver.find_element("xpath",'''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/
section[2]/div[2]/ul/li[2]/span
''')

driver.execute_script("arguments[0].click()", botao_aba)

In [16]:
#verificar se o numero total de paginas continua o mesmo
numero_paginas = driver.find_element("xpath",'''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]
''')
numero_paginas = int(numero_paginas.text.replace("of",""))

#Voltar pra pagina 1 e fazer o mesmo loop anteriormente
for i in range(numero_paginas):   
    voltar = driver.find_element("xpath",'''
    /html/body/div[5]/section/div/div[3]/section/div/div/
    div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[1]
    ''')
    driver.execute_script("arguments[0].click()", voltar)

tabelas_p = []

for i in range(0,numero_paginas):
    html_tabela = driver.find_element("xpath", '''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/div/table
''')

    html_tabela = html_tabela.get_attribute("outerHTML")
#Agora leremos HTML com pandas :D
    tabela = pd.read_html(html_tabela)[0] #recortar primeira p ignorar outros elementos html
    tabelas_p.append(tabela)
#Encontrar o botao de avancar pagina e clicar
    avancar = driver.find_element("xpath", "/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]")
    driver.execute_script("arguments[0].click();", avancar)

In [17]:
tabelas_p = pd.concat(tabelas_p)

tabelas_p

,Ticker,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
0,SPY,SPDR S&P 500 ETF Trust,4.96%,9.32%,6.45%,-9.58%,8.73%,11.15%,12.44%,02/09/23
1,IVV,iShares Core S&P 500 ETF,5.01%,9.33%,6.47%,-9.55%,8.74%,11.17%,12.50%,02/09/23
2,VTI,Vanguard Total Stock Market ETF,5.42%,9.66%,7.03%,-10.06%,8.29%,10.66%,12.08%,02/09/23
3,VOO,Vanguard S&P 500 ETF,4.99%,9.31%,6.48%,-9.57%,8.71%,11.17%,12.50%,02/09/23
4,QQQ,Invesco QQQ Trust,11.51%,14.85%,13.29%,-17.13%,10.27%,14.91%,17.15%,02/09/23
...,...,...,...,...,...,...,...,...,...,...
8,CVSB,Calvert Ultra-Short Investment Grade ETF,--,--,--,--,--,--,--,02/09/23
9,CVLC,Calvert US Large-Cap Core Responsible Index ETF,--,--,--,--,--,--,--,02/09/23
10,CVMC,Calvert US Mid-Cap Core Responsible Index ETF,--,--,--,--,--,--,--,02/09/23
11,CVIE,Calvert International Responsible Index ETF,--,--,--,--,--,--,--,02/09/23


In [18]:
# Exercicio: Buscar uma terceira tabela com todas as paginas da aba Fundamentals

#botao Fundamentals
botao_aba = driver.find_element("xpath",'''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[5]/span
''')

driver.execute_script("arguments[0].click()", botao_aba)

#verificar se o numero total de paginas continua o mesmo
numero_paginas = driver.find_element("xpath",'''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]
''')
numero_paginas = int(numero_paginas.text.replace("of",""))

#Voltar pra pagina 1 e fazer o mesmo loop anteriormente
for i in range(numero_paginas):   
    voltar = driver.find_element("xpath",'''
    /html/body/div[5]/section/div/div[3]/section/div/div/
    div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[1]
    ''')
    driver.execute_script("arguments[0].click()", voltar)

tabelas_f = []

for i in range(0,numero_paginas):
    html_tabela = driver.find_element("xpath", '''
/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/div/table
''')

    html_tabela = html_tabela.get_attribute("outerHTML")
#Agora leremos HTML com pandas :D
    tabela = pd.read_html(html_tabela)[0] #recortar primeira p ignorar outros elementos html
    tabelas_f.append(tabela)
#Encontrar o botao de avancar pagina e clicar
    avancar = driver.find_element("xpath", "/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/span[2]")
    driver.execute_script("arguments[0].click();", avancar)

tabelas_f = pd.concat(tabelas_f)

In [19]:
## Juntando tabela de cadastros com a de rentabilidade (tabelas e tabelas_p):

#Colocar o ticker como indeces, inves dos numeros de linhas

tabelas = tabelas.set_index("Ticker")
tabelas_p = tabelas_p.set_index("Ticker")
#tabelas_f = tabelas_f.set_index("Ticker") dando erro, ver dps

In [20]:
#Adicionar as informações da rentabilidade em cada ano à tabela inicial
tabelas_p[['1 Year', '3 Years', '5 Years','10 Years']]

,1 Year,3 Years,5 Years,10 Years
Ticker,,,,
SPY,-9.58%,8.73%,11.15%,12.44%
IVV,-9.55%,8.74%,11.17%,12.50%
VTI,-10.06%,8.29%,10.66%,12.08%
VOO,-9.57%,8.71%,11.17%,12.50%
QQQ,-17.13%,10.27%,14.91%,17.15%
...,...,...,...,...
CVSB,--,--,--,--
CVLC,--,--,--,--
CVMC,--,--,--,--


In [21]:
tabelas = tabelas.join(tabelas_p[['1 Year', '3 Years', '5 Years','10 Years']])
tabelas

,Name,Segment,Issuer,Expense Ratio,AUM,1 Year,3 Years,5 Years,10 Years
Ticker,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.11B,-9.58%,8.73%,11.15%,12.44%
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$310.04B,-9.55%,8.74%,11.17%,12.50%
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$284.32B,-10.06%,8.29%,10.66%,12.08%
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$282.26B,-9.57%,8.71%,11.17%,12.50%
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$161.85B,-17.13%,10.27%,14.91%,17.15%
...,...,...,...,...,...,...,...,...,...
CVSB,Calvert Ultra-Short Investment Grade ETF,"Fixed Income: Global - Broad Market, Broad-bas...",Morgan Stanley,0.24%,--,--,--,--,--
CVLC,Calvert US Large-Cap Core Responsible Index ETF,Equity: U.S. - Large Cap,Morgan Stanley,--,--,--,--,--,--
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,Equity: U.S. - Mid Cap,Morgan Stanley,0.15%,--,--,--,--,--


- Isso é possível pois possuem os mesmos indeces! Nesse caso, o ticker de cada ETF

In [22]:
# Fechando o navegador
driver.quit()